Visualization time (GRAPH TIME). vente med at sige at "30P001.HMI.DATA_2" and
"30P002.HMI.DATA_2" er binary og skal være line graph med sharp edges?
the variable is called VarValue and time is called TimeString in the csv

##############################################################

When new unitype is shown then the graph should add other vertical axis (vi skal lige starte et sted her)
Units kan ses inde i: "Correlation between sensors and stored data" filen

Spørg gpt om hvor vi kunne informere om unit type ift at danne nye vertikale akser når en ny unit vises i grafen.

In [ ]:
import os
import pickle
from datetime import datetime
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px

app = dash.Dash(__name__)

# Directory paths
directory = "C:/Users/peter/Desktop/Speciale/P10. 2024. Pilot dashboard (1)"

# List of variable names to filter
variable_names = [
    "AI Values_78TT001 - Analog input",
    "AI Values_78TT002 - Analog input",
    "AI Values_10TT002 - Analog input",
    "AI Values_20TTC001 - Analog input",
    "AI Values_20FTC003 - analog input",
    "AI Values_78FT001 - Analog input",
    "AI Values_20FTC002 - Analog input",
    "AI Values_20XTC001 - Analog input",
    "AI Values_20XTC002 - Analog input",
    "AI Values_20XT004 - Analog input",
    "AI Values_20XTC003 - Analog input",
    "AI Values_10PT001 - Analog input",
    "30P001.HMI.DATA_2",
    "30P002.HMI.DATA_2",
    "30P001.HMI.STATUS",
    "AO Values_10R001",
    "AO Values_10R002",
    "AO Values_10R003",
    "AI Values_20PT004 - Analog input",
    "AI Values_78PT002 - Analog input",
    "AI Values_78PT001 - Analog input",
]

# Store filename prefixes in memory
filename_prefixes = []

# Dictionary to store merged DataFrames for all variables
merged_dataframes = {}  # This will have prefixes as keys

# File to store saved prefixes and merged dataframes
prefix_file = "saved_prefixes.txt"
dataframes_file = "merged_dataframes.pkl"

# List CSV files in the directory
def list_csv_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.csv')]

# Filter files by date
def filter_files_by_date(files, start_date, end_date):
    filtered_files = []
    for file in files:
        try:
            file_date_str = file.split('_')[2]  # Extract date from filename
            file_date = datetime.strptime(file_date_str, "%Y%m%d")
            if start_date <= file_date <= end_date:
                filtered_files.append(file)
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    return filtered_files

# Process CSV files to extract variables
def process_csv_file(directory, csv_file, variable_names):
    os.chdir(directory)
    df = pd.read_csv(csv_file, delimiter=';', on_bad_lines='skip')
    dataframes = {}
    for var_name in variable_names:
        filtered_df = df[df['VarName'] == var_name]
        if not filtered_df.empty:
            dataframes[var_name] = filtered_df
    return dataframes

# Save prefixes to a file
def save_prefixes_to_file():
    with open(prefix_file, 'w') as f:
        for prefix in filename_prefixes:
            f.write(f"{prefix}\n")

# Load prefixes from a file
def load_prefixes_from_file():
    if os.path.exists(prefix_file):
        with open(prefix_file, 'r') as f:
            return [line.strip() for line in f.readlines()]
    return []

# Save merged dataframes to a file (pickled)
def save_dataframes_to_file():
    with open(dataframes_file, 'wb') as f:
        pickle.dump(merged_dataframes, f)

# Load merged dataframes from a file (unpickle)
def load_dataframes_from_file():
    if os.path.exists(dataframes_file):
        with open(dataframes_file, 'rb') as f:
            return pickle.load(f)
    return {}

# Load prefixes and merged dataframes when the app starts
filename_prefixes = load_prefixes_from_file()
merged_dataframes = load_dataframes_from_file()

# Dash layout
app.layout = html.Div([
    # Dropdowns for selecting prefix and variables
    html.Div([
        dcc.Dropdown(id='prefix-dropdown', placeholder='Select saved file prefix', style={'width': '50%', 'marginBottom': '10px'}, multi=True),
        dcc.Dropdown(id='variable-dropdown', placeholder='Select variable', style={'width': '50%', 'marginBottom': '10px'}, multi=True)
    ], style={'display': 'flex', 'flexDirection': 'column', 'alignItems': 'center', 'justifyContent': 'center', 'width': '100%', 'marginBottom': '20px'}),

    # Define Batch section and file list
    html.Div([
        html.Div([
            html.H3('Define Batch', style={'marginBottom': '10px'}),
            dcc.DatePickerRange(id='date-picker-range', start_date=datetime(2024, 1, 1), end_date=datetime(2024, 12, 31), display_format='YYYY-MM-DD', style={'marginBottom': '10px', 'width': '100%'}),
            dcc.Input(id='filename-input', type='text', placeholder='Enter filename prefix', style={'display': 'block', 'margin': '10px 0', 'width': '19%'}),
            html.Button('Confirm', id='confirm-button', n_clicks=0, style={'display': 'block', 'margin': '10px 0'}),
            html.Div(id='file-list', style={'whiteSpace': 'pre-line', 'marginTop': '10px'}),
            html.Div(id='save-status', style={'marginTop': '10px'})
        ], style={'textAlign': 'left', 'width': '30%', 'padding': '20px', 'flex': '1'}),

        # Graph to visualize data
        html.Div([dcc.Graph(id='variable-graph')], style={'textAlign': 'center', 'width': '70%', 'padding': '20px', 'flex': '2'})
    ], style={'display': 'flex', 'width': '100%', 'marginTop': '30px'})
], style={'width': '100%', 'padding': '20px'})

# Callback to handle file selection and processing (but no saving)
@app.callback(
    Output('file-list', 'children'),
    Output('prefix-dropdown', 'options'),
    [Input('confirm-button', 'n_clicks')],
    [State('date-picker-range', 'start_date'), State('date-picker-range', 'end_date'), State('filename-input', 'value')]
)
def update_file_list(n_clicks, start_date, end_date, filename_prefix):
    if n_clicks > 0 and start_date and end_date and filename_prefix:
        start_date = datetime.fromisoformat(start_date)
        end_date = datetime.fromisoformat(end_date)
        csv_files = list_csv_files(directory)
        selected_files = filter_files_by_date(csv_files, start_date, end_date)

        if not selected_files:
            return "No files selected.", []

        global merged_dataframes
        merged_dataframes[filename_prefix] = {var: pd.DataFrame() for var in variable_names}  # Initialize for this prefix

        for file in selected_files:
            csv_file_path = os.path.join(directory, file)
            dataframes = process_csv_file(directory, csv_file_path, variable_names)
            for var_name, df in dataframes.items():
                if not df.empty:
                    merged_dataframes[filename_prefix][var_name] = pd.concat([merged_dataframes[filename_prefix][var_name], df], ignore_index=True)

        # Update the prefix dropdown options
        if filename_prefix not in filename_prefixes:
            filename_prefixes.append(filename_prefix)
            save_prefixes_to_file()  # Save the updated prefixes to the file
            save_dataframes_to_file()  # Save the updated merged dataframes to the file

        prefix_options = [{'label': prefix, 'value': prefix} for prefix in filename_prefixes]

        return "\n".join(selected_files), prefix_options
    
    return "No files selected.", [{'label': prefix, 'value': prefix} for prefix in filename_prefixes]

# Callback to update the variable dropdown based on selected prefixes
@app.callback(
    Output('variable-dropdown', 'options'),
    [Input('prefix-dropdown', 'value')]
)
def update_variable_dropdown(selected_prefixes):
    if selected_prefixes:
        variable_options = []
        for prefix in selected_prefixes:
            if prefix in merged_dataframes:  # Ensure prefix exists in merged data
                for var_name in merged_dataframes[prefix]:
                    variable_options.append({'label': f"{prefix}_{var_name}", 'value': f"{prefix}_{var_name}"})
        return variable_options  # Return all variables for the selected prefixes
    return []

# Callback to update the graph based on selected variables
@app.callback(
    Output('variable-graph', 'figure'),
    [Input('variable-dropdown', 'value')]
)
def update_graph(selected_variables):
    if selected_variables:
        all_data = pd.DataFrame()

        for var in selected_variables:
            prefix, var_name = var.split('_', 1)
            if prefix in merged_dataframes and var_name in merged_dataframes[prefix]:
                df = merged_dataframes[prefix][var_name]
                if 'TimeString' in df.columns and 'VarValue' in df.columns:
                    df['TimeString'] = pd.to_datetime(df['TimeString'])
                    df['Variable'] = var_name
                    all_data = pd.concat([all_data, df], ignore_index=True)

        if not all_data.empty:
            fig = px.line(all_data, x='TimeString', y='VarValue', color='Variable', title="Selected Variable Data Over Time")
            return fig

    return px.line()

# Run the Dash server
if __name__ == '__main__':
    app.run_server(debug=True, port=8069)